# Restoring the Scheduler

### 1. Read the queue

In [4]:
with open('res/dialup-log-1985.txt', 'r') as f:
    data = f.readlines()

# Déterminons l´index où l´on trouve 'Synchronised.'
# On ne garde que les lignes après cette ligne
start_index = 0
for i, line in enumerate(data):
    if 'Synchronised.' in line:
        start_index = i + 1
        break
data = data[start_index:]
print(f"Nombre de lignes après 'Synchronised.': {len(data)}")

# Filtrons les lignes ne contenant que 'RECV MSG: 0x'
messages_recus = list(filter(lambda line: all(['RECV MSG: 0x' in line, 'duplicate' not in line, 'resend' not in line]), data))
print(f"Nombre de lignes après filtrage des messages reçus: {len(messages_recus)}")

Nombre de lignes après 'Synchronised.': 41
Nombre de lignes après filtrage des messages reçus: 32


### 2. Decode the headers

In [42]:


from datetime import datetime


class Message:
    def __init__(self, message: str) -> None:
        self._message = message
        self.date, self.header = self.decrypte_hex_line(message)
        self.binary_header = format(int(self.header, 16), '032b')

    def decrypte_hex_line(self, line: str) -> tuple[datetime, str]:
        date, _, _, hex_header = tuple(filter(None, line.strip().split(' ')))
        return datetime.strptime(date, '%H:%M:%S'), hex_header
    
    @property
    def priority(self) -> str:
        return self.binary_header[-3:]
    
    @property
    def timestamp(self) -> str:
        return self.binary_header[-16:-3]
    
    @property
    def aging_factor(self) -> str:
        return self.binary_header[-20:-16]
    
    @property
    def payload(self) -> str:
        return self.binary_header[:-20]
    
def convert_32bit_to_hex(bits) -> str:
    return hex(int(bits, 2))
    
    
decoded_messages = [Message(line) for line in messages_recus]
# Excluons les messages avec un timestamp 0x1FFF
decoded_messages = [msg for msg in decoded_messages if convert_32bit_to_hex(msg.timestamp) != '0x1fff']
print(f"Nombre de messages après exclusion des timestamps 0x1FFF: {len(decoded_messages)}")


Nombre de messages après exclusion des timestamps 0x1FFF: 28


### 3. Repair corrupted headers